In [3]:
import os
import numpy as np
import pandas as pd
import zipfile
import shutil
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [5]:
# Unzip train.zip
with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall("train")

# Unzip test.zip
with zipfile.ZipFile("test.zip", 'r') as zip_ref:
    zip_ref.extractall("test")

In [6]:
labels_df = pd.read_csv('labels.csv')

# Encode breeds
lb = LabelBinarizer()
labels_df['breed_encoded'] = list(lb.fit_transform(labels_df['breed']))
labels_df['path'] = labels_df['id'].apply(lambda x: f"train/{x}.jpg")

# Parameters
img_size = 224
batch_size = 32

# Generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.1)

train_gen = datagen.flow_from_dataframe(
    labels_df,
    x_col='path',
    y_col='breed',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_dataframe(
    labels_df,
    x_col='path',
    y_col='breed',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(lb.classes_)

Found 9200 validated image filenames belonging to 120 classes.
Found 1022 validated image filenames belonging to 120 classes.


In [7]:
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        30,840 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,616,760 (9.98 MB)

 Trainable params: 358,776 (1.37 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [9]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stop]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 628s 2s/step - accuracy: 0.3628 - loss: 2.8537 - val_accuracy: 0.7554 - val_loss: 0.8151
Epoch 2/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 613s 2s/step - accuracy: 0.7597 - loss: 0.7967 - val_accuracy: 0.7926 - val_loss: 0.7315
Epoch 3/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 614s 2s/step - accuracy: 0.8304 - loss: 0.5373 - val_accuracy: 0.7798 - val_loss: 0.7520
Epoch 4/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 610s 2s/step - accuracy: 0.8784 - loss: 0.3848 - val_accuracy: 0.7945 - val_loss: 0.7705
Epoch 5/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 608s 2s/step - accuracy: 0.8948 - loss: 0.3258 - val_accuracy: 0.7818 - val_loss: 0.7349


In [10]:
model.save("best_model_earlystop.h5")

In [11]:
test_files = os.listdir("test")
test_ids = [f[:-4] for f in test_files if f.endswith(".jpg")]

# Preprocess test images
test_images = []
for image_id in tqdm(test_ids):
    img_path = f"test/{image_id}.jpg"
    img = Image.open(img_path).resize((img_size, img_size))
    img = np.array(img)
    if img.shape != (img_size, img_size, 3):
        continue
    img = preprocess_input(img)
    test_images.append(img)

test_images = np.array(test_images)

100%|██████████| 10357/10357 [00:40<00:00, 254.68it/s]


In [12]:
preds = model.predict(test_images, verbose=1)
submission = pd.DataFrame(preds, columns=lb.classes_)
submission.insert(0, 'id', test_ids[:len(submission)])
submission.to_csv('submission1.csv', index=False)

324/324 ━━━━━━━━━━━━━━━━━━━━ 617s 2s/step


In [14]:
from google.colab import files
files.download("submission1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>